In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import ddsp
from thesis import newt

In [ ]:
import gin
gin.enter_interactive_mode()

In [ ]:
harm = newt.NEWTHarmonic(n_harmonics=1, n_outputs=3, sample_rate=16000)

f0 = tf.zeros((2, 16000)) + 440
# f0 = tf.linspace(440, 880, 16000).reshape((1, -1))
res = harm(f0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(res[0,:100])

In [ ]:
import IPython.display
IPython.display.display(IPython.display.Audio(res[0,:,0], rate=16000))

In [ ]:
x = res[0,:,0]
mask = (x[1:-1] > x[:-2]) & (x[1:-1] > x[2:])
sum(mask)

In [ ]:
plt.plot(mask[:100])
plt.plot(x[:100])
